In [1]:
from google.cloud import bigquery
import pandas as pd 
import chime   
import seaborn as sns
client = bigquery.Client("gcp-wow-rwds-ai-checkout-dev")

/opt/homebrew/lib/python3.11/site-packages/google/auth/_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', None)

Search Related Assets:
* Test = Used a specific keyword, saw a specific asset on a specific page
* Control = Used the same keyword, saw the same page, did not see the campaign asset

In [10]:
check = client.query(
"""
CREATE OR REPLACE TABLE gcp-wow-cart-data-dev-d4d7.davide.dx_version_four_eda_1 AS 
SELECT  
    udo_page_name,
    udo_page_type,
    udo_app_section,
    udo_acs_stitch_id,
    CASE 
        WHEN udo_tealium_datasource IN ('kmm8nj', '7n58ym') THEN 'App'
        WHEN udo_tealium_datasource = 'qurl3t' THEN 'Web'
    END AS platform,
    SPLIT(udo_acs_stitch_id, ":")[OFFSET(2)] AS tealium_media_type,
    LOWER(TRIM(REPLACE(REGEXP_REPLACE(udo_search_keyword_5172,"[^a-zA-Z0-9 &-'@]", ""),'\t',''))) search_term, 
    MIN(DATE(post_time, "Australia/Sydney")) AS earliest_visit,
    MAX(DATE(post_time, "Australia/Sydney")) AS latest_visit,
    COUNT(DISTINCT visitor_id) AS n_visitors,
    COUNT(DISTINCT CASE 
        WHEN SPLIT(udo_acs_stitch_id, ":")[SAFE_OFFSET(0)] = dim_campaign.booking_number 
        AND SPLIT(udo_acs_stitch_id, ":")[SAFE_OFFSET(3)] = dim_campaign.line_name 
        THEN visitor_id 
    END) AS n_visitors_exposed,
    COUNT(DISTINCT CASE 
        WHEN SPLIT(udo_acs_stitch_id, ":")[SAFE_OFFSET(0)] <> dim_campaign.booking_number 
        AND SPLIT(udo_acs_stitch_id, ":")[SAFE_OFFSET(3)] <> dim_campaign.line_name 
        THEN visitor_id 
    END) AS n_visitors_not_exposed
FROM `gcp-wow-rwds-ai-mlt-evs-prod.event_store.tealium_events` all_events
LEFT JOIN `gcp-wow-cart-data-prod-d710.cdm.dim_cartology_campaigns` dim_campaign 
    ON TRIM(SPLIT(udo_acs_stitch_id, ":")[SAFE_OFFSET(0)]) = TRIM(dim_campaign.booking_number)
    AND TRIM(SPLIT(udo_acs_stitch_id, ":")[SAFE_OFFSET(3)]) = TRIM(dim_campaign.line_name)
INNER JOIN `gcp-wow-cart-data-dev-d4d7.davide.multishare_pack_bookings` multishare 
    ON multishare.string_field_0 = TRIM(SPLIT(udo_acs_stitch_id, ":")[SAFE_OFFSET(0)])
WHERE 
    1 = 1 
    AND DATE(post_time, "Australia/Sydney") >= DATE("2024-02-01")
    AND DATE(post_time, "Australia/Sydney") < DATE("2024-03-01")
    AND ARRAY_LENGTH(SPLIT(udo_acs_stitch_id, ":")) >= 4
    AND udo_acs_stitch_id IS NOT NULL
    AND udo_tealium_datasource IN ("qurl3t", 'kmm8nj', '7n58ym')
GROUP BY 1,2,3,4,5,6,7
ORDER BY 8 DESC;
"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df 

""


In [29]:
check = client.query(
"""
SELECT * FROM gcp-wow-cart-data-dev-d4d7.davide.dx_version_four_eda_1 LIMIT 10;
"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df 

,udo_page_name,udo_page_type,udo_app_section,udo_acs_stitch_id,platform,tealium_media_type,search_term,earliest_visit,latest_visit,n_visitors,n_visitors_exposed,n_visitors_not_exposed
0,None,None,Products,WOW20008499:wow-reach-all---app-banners:banner-category:CI-10096351:0054999:app,App,banner-category,milk chocolate hearts,2024-02-29,2024-02-29,1,1,0
1,ww-sm:shop:browse:dairy-eggs-fridge:vegetarian-vegan:vegetarian-meat-free,productcategory,None,WOW20007365:category-web-content-card---vegetarian---plant:content-card-category:CI-10092448:0054406:web,Web,content-card-category,impossible meat,2024-02-29,2024-02-29,1,0,0
2,ww-sm:shop:browse:pantry:long-life-milk:almond-other-nut-milks,productcategory,None,WOW20006592:wow-reach-grocery---content-card:content-card-search:CI-10106941:0054744:web,Web,content-card-search,None,2024-02-29,2024-02-29,2,2,0
3,ww-sm:shop:list:home,list,None,WOW20007365:promo-tile---dairy-eggs-and-fridge:promo-tile:CI-10092446:0055283:web,Web,promo-tile,None,2024-02-29,2024-02-29,2,2,0
4,None,None,Products,WOW20009070:search-app-banner-2---mens-care:banner-category:CI-10106074:0055226:app,App,banner-category,clippers,2024-02-29,2024-02-29,1,1,0
5,None,None,Products,WOW20009703:wow-reach-all---app-banners:banner-category:CI-10110011:0055091:app,App,banner-category,lactose free natural cheese,2024-02-29,2024-02-29,1,1,0
6,ww-sm:shop:recipes:collections:category:soup:chicken-soup,recipecategory,None,WOW20006914:wow-reach-grocery---leaderboard:search-display-banner:CI-10086085:0054526:web,Web,search-display-banner,None,2024-02-29,2024-02-29,1,1,0
7,None,None,Products,WOW20008499:wow-reach-all---app-banners:banner-category:CI-10096351:0054999:app,App,banner-category,olives,2024-02-29,2024-02-29,1,1,0
8,None,None,Products,WOW20009703:wow-reach-all---app-banners:banner-category:CI-10110011:0055091:app,App,banner-category,toilet block cleaner,2024-02-29,2024-02-29,1,1,0
9,None,None,Products,WOW20009703:wow-reach-all---app-banners:banner-category:CI-10110011:0055091:app,App,banner-category,cacao,2024-02-29,2024-02-29,2,2,0


In [31]:
check = client.query(
"""
CREATE OR REPLACE TABLE gcp-wow-cart-data-dev-d4d7.davide.dx_version_four_test_group AS (
    with reach_all AS (
        SELECT 
            SPLIT(udo_acs_stitch_id, ":")[SAFE_OFFSET(0)] AS tealium_booking_id, 
            COUNT(DISTINCT CASE WHEN udo_acs_stitch_id LIKE "%reach-all%" THEN SPLIT(udo_acs_stitch_id, ":")[SAFE_OFFSET(0)] END) AS n_reach_all_assets,
            COUNT(DISTINCT CASE WHEN udo_acs_stitch_id NOT LIKE "%reach-all%" THEN SPLIT(udo_acs_stitch_id, ":")[SAFE_OFFSET(3)] END) AS n_not_reach_all_assets
        FROM gcp-wow-cart-data-dev-d4d7.davide.dx_version_four_eda_1
        GROUP BY 1
        ORDER BY 2
    ),
    is_asset_search_related AS (
        SELECT 
            SPLIT(eda.udo_acs_stitch_id, ":")[SAFE_OFFSET(0)] AS booking_id, 
            platform,
            udo_page_name, 
            udo_page_type, 
            udo_app_section,
            tealium_media_type, 
            COUNT(DISTINCT search_term) AS n_search_terms,
            SUM(n_visitors) AS total_visitors
        FROM gcp-wow-cart-data-dev-d4d7.davide.dx_version_four_eda_1 eda
        LEFT JOIN (SELECT DISTINCT tealium_booking_id FROM reach_all WHERE n_reach_all_assets > 0) reach
            ON reach.tealium_booking_id = SPLIT(eda.udo_acs_stitch_id, ":")[SAFE_OFFSET(0)]
        WHERE reach.tealium_booking_id IS NULL
        GROUP BY ALL
        ORDER BY 1,2 
    )

    SELECT    
        udo_page_name,
        udo_page_type,
        udo_app_section,
        udo_acs_stitch_id,
        CASE WHEN udo_tealium_datasource IN ('kmm8nj', '7n58ym') THEN 'App' WHEN udo_tealium_datasource = 'qurl3t' THEN 'Web' END AS platform,
        SPLIT(udo_acs_stitch_id, ":")[OFFSET(2)] AS tealium_media_type,
        all_events.visitor_id,
        shopper_id,
        crn,
        ARRAY_AGG(DISTINCT LOWER(TRIM(REPLACE(REGEXP_REPLACE(udo_search_keyword_5172,"[^a-zA-Z0-9 &-'@]", ""),'\t',''))) IGNORE NULLS) search_terms
    FROM `gcp-wow-rwds-ai-mlt-evs-prod.event_store.tealium_events` all_events
    
    -- Campaign Details
    LEFT JOIN `gcp-wow-cart-data-prod-d710.cdm.dim_cartology_campaigns` dim_campaign 
        ON TRIM(SPLIT(udo_acs_stitch_id, ":")[SAFE_OFFSET(0)]) = TRIM(dim_campaign.booking_number)
        AND TRIM(SPLIT(udo_acs_stitch_id, ":")[SAFE_OFFSET(3)]) = TRIM(dim_campaign.line_name)
    
    -- Specific Campaigns Chris asked for
    INNER JOIN (SELECT DISTINCT string_field_0 FROM `gcp-wow-cart-data-dev-d4d7.davide.multishare_pack_bookings`) multishare 
        ON multishare.string_field_0 = TRIM(SPLIT(udo_acs_stitch_id, ":")[SAFE_OFFSET(0)])

    -- Exclude 'reach-all'
    LEFT JOIN (SELECT DISTINCT tealium_booking_id FROM reach_all WHERE n_reach_all_assets > 0) reach
        ON reach.tealium_booking_id = SPLIT(all_events.udo_acs_stitch_id, ":")[SAFE_OFFSET(0)]
            
    -- shopper id
    LEFT JOIN `gcp-wow-rwds-ai-mlt-evs-prod.event_store.tealium_visitorid_shopperid_map_window` shopper_id_map
        ON all_events.visitor_id = shopper_id_map.visitor_id
        AND all_events.post_time BETWEEN shopper_id_map.effective_time AND shopper_id_map.expiry_time
        
    -- crn
    LEFT JOIN `gcp-wow-rwds-ai-mlt-evs-prod.event_store.tealium_visitorid_crn_map_window` crn_map
        ON all_events.visitor_id = crn_map.visitor_id
        AND all_events.post_time BETWEEN crn_map.effective_time AND crn_map.expiry_time

    WHERE 
        DATE(post_time, "Australia/Sydney") >= DATE("2024-02-01")
        AND DATE(post_time, "Australia/Sydney") < DATE("2024-03-01")
        AND ARRAY_LENGTH(SPLIT(udo_acs_stitch_id, ":")) >= 4
        AND udo_acs_stitch_id IS NOT NULL
        AND all_events.post_time BETWEEN shopper_id_map.effective_time AND shopper_id_map.expiry_time
        AND all_events.post_time BETWEEN crn_map.effective_time AND crn_map.expiry_time
        AND udo_tealium_datasource IN ("qurl3t", 'kmm8nj', '7n58ym')
        AND crn_map.crn IS NOT NULL
        AND reach.tealium_booking_id IS NULL
        AND all_events.visitor_id IS NOT NULL
    GROUP BY ALL
);

"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df 

""


In [33]:
check = client.query(
"""
SELECT * FROM gcp-wow-cart-data-dev-d4d7.davide.dx_version_four_test_group WHERE array_length(search_terms) > 1 LIMIT 10
"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df 

,udo_page_name,udo_page_type,udo_app_section,udo_acs_stitch_id,platform,tealium_media_type,visitor_id,shopper_id,crn,search_terms
0,None,None,Products,WOW20006484:search-app-banner-2---meat:banner-category:CI-10099077:0051343:app,App,banner-category,c23002c30d03486797717e78e0f82498,11552502,3300000000007826960,"[meat, hot dogs]"
1,None,None,Products,WOW20006484:search-app-banner-2---meat:banner-category:CI-10099077:0051343:app,App,banner-category,475b904376bb4d249f218d8ca8638e44,7878123,1100000000088197420,"[roast, bacon]"
2,None,None,Products,WOW20006484:search-app-banner-2---meat:banner-category:CI-10099077:0051343:app,App,banner-category,caac0b94014e4450abc3a839ab123eb6,2963876,1100000000136518347,"[mince, streaky bacon]"
3,None,None,Products,WOW20006484:search-app-banner-2---meat:banner-category:CI-10099077:0051343:app,App,banner-category,d0280fc22dc144f291489884ef48d5df,4546987,1100000000000269700,"[mince, curried sausages]"
4,None,None,Products,WOW20006484:search-app-banner-2---meat:banner-category:CI-10099077:0051343:app,App,banner-category,4e906101819040d5a119a065b48f943f,5361064,1100000000130882717,"[bacon, mince]"
5,None,None,Products,WOW20006484:search-app-banner-2---meat:banner-category:CI-10099077:0051343:app,App,banner-category,5a367cfbea0c405f9badc35c18f62ef0,6032192,3300000000002867620,"[schnitzel, kiev]"
6,None,None,Products,WOW20006484:search-app-banner-2---meat:banner-category:CI-10099077:0051343:app,App,banner-category,549dfabc5a654520b5b15ca40f409f38,6301199,3300000000003159496,"[bacon, mince]"
7,None,None,Products,WOW20006484:search-app-banner-2---meat:banner-category:CI-10099077:0051343:app,App,banner-category,99c3aff2f7d94d3db79b2f1d398f999c,2939332,1100000000003458248,"[mince, roast]"
8,None,None,Products,WOW20006484:search-app-banner-2---meat:banner-category:CI-10099077:0051343:app,App,banner-category,c882b480657d4c11913838fc7eb97409,8218296,3300000000000733992,"[grass, bacon]"
9,None,None,Products,WOW20006484:search-app-banner-2---meat:banner-category:CI-10099077:0051343:app,App,banner-category,57ff4521c1b84888b4a3972c0f6247eb,4879111,1100000000002543493,"[bacon, schnitzel]"


In [ ]:
check = client.query(
"""

CREATE OR REPLACE TABLE gcp-wow-cart-data-dev-d4d7.davide.dx_version_four_control_group AS (
control_group AS (
    SELECT DISTINCT 
        control.visitor_id,
        control.shopper_id 
    FROM (
        SELECT  
            udo_page_name,
            udo_page_type,
            udo_app_section,
            udo_acs_stitch_id,
            CASE WHEN udo_acs_stitch_id IS NOT NULL THEN  SPLIT(udo_acs_stitch_id, ":")[OFFSET(0)]  ELSE NULL END AS booking_id, 
            CASE WHEN udo_acs_stitch_id IS NOT NULL THEN  SPLIT(udo_acs_stitch_id, ":")[OFFSET(3)]  ELSE NULL END AS asset_id, 
            LOWER(TRIM(REPLACE(REGEXP_REPLACE(udo_search_keyword_5172,"[^a-zA-Z0-9 &-'@]", ""),'\t',''))) AS search_term,
            CASE 
                WHEN udo_tealium_datasource IN ('kmm8nj', '7n58ym') THEN 'App'
                WHEN udo_tealium_datasource = 'qurl3t' THEN 'Web'
            END AS platform,
            CASE WHEN udo_acs_stitch_id IS NOT NULL THEN SPLIT(udo_acs_stitch_id, ":")[OFFSET(2)] ELSE NULL END AS tealium_media_type,
            all_events.visitor_id,
            shopper_id,
            crn
        FROM `gcp-wow-rwds-ai-mlt-evs-prod.event_store.tealium_events` all_events
        LEFT JOIN `gcp-wow-cart-data-prod-d710.cdm.dim_cartology_campaigns` dim_campaign 
            ON TRIM(SPLIT(udo_acs_stitch_id, ":")[SAFE_OFFSET(0)]) = TRIM(dim_campaign.booking_number)
            AND TRIM(SPLIT(udo_acs_stitch_id, ":")[SAFE_OFFSET(3)]) = TRIM(dim_campaign.line_name)

        INNER JOIN (SELECT DISTINCT TRIM(SPLIT(udo_acs_stitch_id, ":")[SAFE_OFFSET(0)]) AS booking FROM gcp-wow-cart-data-dev-d4d7.davide.dx_version_four_test_group) test_group
            ON test_group.booking = SPLIT(udo_acs_stitch_id, ":")[OFFSET(0)]

        -- shopper id
        LEFT JOIN `gcp-wow-rwds-ai-mlt-evs-prod.event_store.tealium_visitorid_shopperid_map_window` shopper_id_map
            ON all_events.visitor_id = shopper_id_map.visitor_id
            AND all_events.post_time BETWEEN shopper_id_map.effective_time AND shopper_id_map.expiry_time
            
        -- crn
        LEFT JOIN `gcp-wow-rwds-ai-mlt-evs-prod.event_store.tealium_visitorid_crn_map_window` crn_map
            ON all_events.visitor_id = shopper_id_map.visitor_id
            AND all_events.post_time BETWEEN shopper_id_map.effective_time AND shopper_id_map.expiry_time

        WHERE 
            1 = 1 
            AND DATE(post_time, "Australia/Sydney") >= DATE("2024-02-01")
            AND DATE(post_time, "Australia/Sydney") < DATE("2024-03-01")
            AND ARRAY_LENGTH(SPLIT(udo_acs_stitch_id, ":")) >= 4
            AND udo_acs_stitch_id IS NOT NULL
            AND udo_tealium_datasource IN ("qurl3t", 'kmm8nj', '7n58ym')
    ) control 
    
    -- Exclude test groups visitors who saw the same page type (web) with no search term 
    LEFT JOIN (SELECT DISTINCT udo_page_name, visitor_id FROM test_group WHERE search_term IS NULL) test_group_page_names_no_search_terms
            ON test_group_page_names_no_search_terms.udo_page_name = control.udo_page_name 
            AND test_group_page_names_no_search_terms.crn = control.crn
         
            AND test_group_page_names_no_search_terms.udo_page_name IS NOT NULL 
            AND control.udo_page_name IS NOT NULL
            AND control.search_term IS NULL

    -- Exclude test groups visitors who saw the same page type (web) using a specific search term 
    LEFT JOIN (SELECT DISTINCT udo_page_name, visitor_id, search_term FROM test_group WHERE search_term IS NOT NULL) test_group_page_names_with_search_terms
            ON test_group_page_names_with_search_terms.udo_page_name = control.udo_page_name 
            AND test_group_page_names_with_search_terms.search_term = control.search_term 
            AND test_group_page_names_with_search_term.crn = control.crn 

            AND control.udo_page_name IS NOT NULL
            AND control.search_term IS NOT NULL
            AND test_group_page_names_with_search_terms.udo_page_name IS NOT NULL 

    -- Exclude test groups visitors who saw the same app section using a specific search term 
    LEFT JOIN (SELECT DISTINCT udo_page_name, visitor_id, search_term FROM test_group WHERE search_term IS NOT NULL AND udo_app_section IS NOT NULL) test_group_app_section_with_search_terms
            ON test_group_page_names_with_search_terms.udo_app_section = control.udo_app_section 
            AND test_group_page_names_with_search_terms.search_term = control.search_term 
            AND test_group_page_names_with_search_term.crn = control.crn 

            AND control.udo_app_section IS NOT NULL
            AND control.search_term IS NOT NULL
            AND test_group_app_section_with_search_terms.udo_app_section IS NOT NULL 

    WHERE 
        test_group_page_names_no_search_terms.crn IS NULL 
    AND 
        test_group_page_names_with_search_terms.crn IS NULL 
    AND 
        test_group_app_section_with_search_terms.crn IS NULL
    AND 
        control.crn IS NOT NULL      
);
"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df 

In [ ]:
check = client.query(
"""
"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df 